# Imports

To refresh imports

In [1]:
%reload_ext autoreload
%autoreload 2

Imports

In [2]:
# -------------------------------- torch stuff ------------------------------- #
import torch
from torch.utils.data import Dataset, DataLoader


# ----------------------------------- other ---------------------------------- #
from glob import glob
# import soundfile as sf
from scipy.io import wavfile
import json
import os
from tqdm import tqdm
import sys
import wandb

# ---------------------------------- Custom ---------------------------------- #
from utils.precision_loss import show_sum_error

### Why using scipy.io instead soundfile

In [3]:
# print("soundfile converts directly to float 64")
# sf.read("../data/LibriCount/10_85b5ac.wav")[0]

In [4]:
print("wavfile concervs the original format : int16")
wavfile.read("../data/LibriCount/10_85b5ac.wav")[1]

wavfile concervs the original format : int16


array([ 3095,  3265,  4328, ..., -1835, -1613, -1282], dtype=int16)

### Format

In [5]:
F16 = torch.float16
F32 = torch.float32
F64 = torch.float64

In [6]:
show_sum_error()

Conversion Errors from int16 in range(-32768, 32768) maxed by (max is=) :
	Float 16  8.0
	Float 32  0.0
	Float 64  0.0


# Consts

In [7]:
data_dir = "../data/LibriCount"
FTYPE = F32

# Dataset

In [8]:
class AudioCountGender(Dataset):
    def __init__(self, data_dir=data_dir, dtype = FTYPE, cache=True):
        self.sounds = glob(os.path.join(data_dir,"*.wav"))
        self.labels = glob(os.path.join(data_dir,"*.json"))
        self.dtype = dtype
        self.cache = cache
        if self.cache:
            self.data = []
            for index in range(len(self.sounds)):
                sample_rate, clip = wavfile.read(self.sounds[index])
                with open(self.labels[index]) as f:
                    label = json.load(f)
                genders = [0, 0] #[Male, Female]
                for person in label:
                    gender = person["sex"]
                    if gender == "F":
                        genders[1] += 1
                    else :
                        genders[0] += 1
                self.data.append([torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype)]) #unsqueeze serves for channel = 1
                
                
    def __getitem__(self, index):
        if self.cache:
            return self.data[index]
        else:
            # clip, sample_rate = sf.read(self.sounds[index])
            sample_rate, clip = wavfile.read(self.sounds[index])
            with open(self.labels[index]) as f:
                label = json.load(f)
            genders = [0, 0] #[Male, Female]
            for person in label:
                gender = person["sex"]
                if gender == "F":
                    genders[1] += 1
                else :
                    genders[0] += 1
            return torch.tensor(clip, dtype=self.dtype).unsqueeze(0), torch.tensor(genders, dtype=self.dtype) #unsqueeze serves for channel = 1
    def __len__(self):
        return len(self.sounds)

In [9]:
data = AudioCountGender()
data[800]

[tensor([[4939., 3212., 1816.,  ..., 7072., 8158., 6819.]]), tensor([4., 3.])]

# Dataloader

In [10]:
BATCH_SIZE = 128
dataloader = DataLoader(dataset=data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Measures

In [11]:
from time import time
# L = []
start = time()
for d in dataloader:
    # L.append(d)
    print(len(d[0]), end="_")
end = time()
print()
print("Duration: {:.2f}s".format(end-start))
#Energy efficient mode
#Duration: 55.97s for F16 and no caching and list appending
#Duration: 1.72s for F16 and caching and list appending
#Equilibre mode 
#Duration: 12.46s for F16 and no caching and no list appending
#Duration: 0.88s for F16 and caching andn no list appending

128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_128_88_
Duration: 0.73s


# Simple model

In [12]:
# torch sequential
c_1 = 64
kernel_1 = 21
c_2 = 64
kernel_2 = 7
kernel_pool = 3

c_3 = 128
kernel_3 = 3
c_4 = 256
kernel_4 = 3
model = torch.nn.Sequential( #input size = 80000
    torch.nn.Conv1d(1, c_1, kernel_1, stride=5, dtype=FTYPE),
    torch.nn.Conv1d(c_1, c_2, kernel_2, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Conv1d(c_2, c_3, kernel_3, dtype=FTYPE),
    torch.nn.Conv1d(c_3, c_4, kernel_4, dtype=FTYPE),
    torch.nn.ReLU(),
    torch.nn.Dropout(),
    torch.nn.MaxPool1d(kernel_pool),
    
    torch.nn.Flatten(),
    torch.nn.Linear(c_4*1775, 2, dtype=FTYPE),
)

In [13]:
example = d[0].to("cuda")
model.to("cuda")

print("Shape : ", model.forward(example).shape)
model.forward(example)

Shape :  torch.Size([88, 2])


tensor([[ 4.2316e+00,  1.5078e+00],
        [ 1.2810e+03,  5.4191e+02],
        [ 1.9512e+00,  1.1733e+01],
        [ 1.9919e+02,  2.4638e+02],
        [-2.7040e+01,  7.7802e+02],
        [ 6.5761e+02, -2.4843e+02],
        [-1.5126e+03,  3.0569e+02],
        [-3.7816e+02,  1.0408e+03],
        [ 7.4865e+02,  9.3248e+02],
        [-1.9008e+00,  4.6438e+00],
        [ 4.1743e+02,  8.2910e+02],
        [-5.3023e+02,  1.1113e+03],
        [-8.6392e+02,  7.2157e+02],
        [ 5.2129e+02,  4.9262e+02],
        [ 5.6131e+02,  4.1689e+02],
        [ 1.4722e+01,  6.2521e+00],
        [-1.3334e+02,  4.5517e+02],
        [-1.6312e+02,  9.1156e+02],
        [ 8.7969e+02,  3.2531e+02],
        [-8.5344e+01,  4.0351e+01],
        [-1.5040e+02,  7.9284e+02],
        [-5.7621e+01,  1.4246e+03],
        [-5.1767e+02,  2.0442e+03],
        [-1.5457e+02,  2.1753e+02],
        [ 1.0335e+03,  8.3827e+02],
        [-1.6789e+01,  1.5439e+01],
        [-8.3639e+02, -2.6937e+01],
        [-4.5168e+02,  2.443

# Wandb

In [14]:
wandb.init(project="klee_project_audio", entity="mustapha")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mustapha (use `wandb login --relogin` to force relogin)


In [15]:
LEARNING_RATE = 0.001
EPOCHS = 700

wandb.config.update({
  "learning_rate": LEARNING_RATE,
  "epochs": EPOCHS,
  "batch_size": BATCH_SIZE
})

# Simple train

In [ ]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
model.to("cuda")

for epoch in range(EPOCHS):
    mean_loss = 0
    for d in tqdm(dataloader):
        audios = d[0].to("cuda")
        labels = d[1].to("cuda")
        predictions = model.forward(audios)
        loss_value = loss(predictions, labels)
        mean_loss += loss_value.item()
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    print("Train Loss : {:.4f}".format(mean_loss/len(dataloader)))
    wandb.log({"loss":mean_loss/len(dataloader), "epoch":epoch})
    # wandb.watch(model)

100%|██████████| 45/45 [00:16<00:00,  2.68it/s]


Epoch 1/700
Train Loss : 1755686615.1381


100%|██████████| 45/45 [00:16<00:00,  2.74it/s]


Epoch 2/700
Train Loss : 9.6927


100%|██████████| 45/45 [00:16<00:00,  2.74it/s]


...

100%|██████████| 45/45 [00:17<00:00,  2.56it/s]


Epoch 100/700
Train Loss : 9.6545


100%|██████████| 45/45 [00:17<00:00,  2.57it/s]


Epoch 101/700
Train Loss : 9.6363


 47%|████▋     | 21/45 [00:08<00:09,  2.58it/s]